### Gathering movie data for approximately 1000 movies

In [60]:
import re
import json 
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from matplotlib import pyplot as plt, rcParams
%matplotlib inline
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

### Source of Movies
In the cell below:
- I gather the urls for lists of the top 100 movies for each genre from rotten tomatoes. 
***
Note: Because I am only scraping movies from a top 100 list from rotten tomatoes, I am definitely gathering a biased selection of movies. Rotten tomatoes doesn't identify how they are ranking these movies as the "Top 100" so that could possibly introduce more bias into the data set. To help fix any bias that may be introduced by collecting my movies this way, I will be collecting a large number of movies, ~1000
***

In [61]:
base_url = "https://www.rottentomatoes.com"
action ="https://www.rottentomatoes.com/top/bestofrt/top_100_action__adventure_movies/"
url_finder = re.compile(r"/top/bestofrt/top_100_.*$")
page_source = requests.get(action).text
time.sleep(1)           # PAUSE before continuing.
soup = BeautifulSoup(page_source, "html.parser")
urls = soup.find_all("a", href = url_finder )
top = []
for url in urls:
    top.append(base_url+url["href"])

### Collecting Movies' Names
In the cell below:
- I go to each url that was gathered in the cell above 
- I get the name of each movie on the list.

In [63]:
all_movies = []
for url in top:
    page_source = requests.get(url).text
    time.sleep(1)  # PAUSE before continuing.
    soup = BeautifulSoup(page_source, "html.parser")
    top_100 = soup.find('script', type='application/ld+json')
    data = json.loads(top_100.text)
    list_dict = data["itemListElement"]
    for top_movies in list_dict:
        page_source = requests.get(top_movies["url"]).text
        time.sleep(.3)
        soup = BeautifulSoup(page_source, "html.parser")
        data = json.loads(soup.find('script', type='application/ld+json').text)
        all_movies.append(data["name"])

### Gathering Information on Each Movie
In the cell below:
- I create a set of the movies name. 
- For each movie I search IMDB for the specific movie, and pick the first result.
- I collect data for "Director", "Writer", "Star", "Release Date", "Budget", "Opening Weekend", "Gross"
- Any missing data is reported as NaN

IMDB is a reputable source for information, according to their website 

>"we [IMDB] actively gather information from and verify items with studios and 
filmmakers".


***  

Because I am picking the first movie result, I may not be picking the exact movie that was listed on rotten tomatoes, which could produce some bias in my dataset. However, it is likely to be the most popular movie searched for, because it will be the first result
***
For each movie I search I collect the first name that appears under "Director", "Writer", and "Star" label. I'm only collecting the first name, because collecting more names can become very complicated very fast. Because I only select the first name, I am relying on IMDB to accurately place the most important person in that category first.
***
I also collect data on the release date, the budget for the movie, the amount of money spent on the movie during the opening weekend, and the gross profit made. 
Unfortunatley all these monetary amounts come in different currencies, so I will have to make this uniform and convert every currency to USD based on the date that the movie was released.
***

In [64]:
all_movies = list(set( all_movies ))
#This movie won't produce any results on IMDB with the parenthesis, which will
#create an error, so this is a quick fix.
oops=all_movies.index("The Good, the Bad, the Weird (Joheun-nom, Nabbeun-nom, Isanghan-nom)")
all_movies[oops] = "The Good, the Bad, the Weird"

n = len(all_movies)
data = [[movie] for movie in all_movies]
base_url = "http://www.imdb.com/"
browser = webdriver.Chrome()
browser.get(base_url)
columns = ["Name","Director","Writer","Star","Release.Date","Budget","Opening.Weekend","Gross"]

for i in range(len(data)):
    # Get the search bar, type in some text, and press Enter.
    search_bar = browser.find_element_by_id("navbar-query")
    search_bar.clear() # Clear any pre-set text.
    search_bar.send_keys(data[i][0])
    search_bar.send_keys(Keys.RETURN) # Press Enter.
    time.sleep(np.random.rand())
    soup = BeautifulSoup(browser.page_source,"html.parser")
    table = soup.find( name = "table" )
    link = table.find_all(name="td")[1]#choose first search result
    url = base_url+link.find(name="a")["href"]
    time.sleep(np.random.rand())
    browser.get(url)#get page for first movie result
    soup = BeautifulSoup(browser.page_source,"html.parser")
    tab = soup.find_all(name="h4")
    D, W, S, B, O, G, R = False, False, False, False, False, False, False
    
    for j in tab:
        if j.text == "Director:" or j.text == "Directors:":
            data[i].append((" ").join(j.next_sibling.next_sibling.text.split()))
            D = True
        if j.text == "Writer:" or j.text == "Writers:":
            data[i].append((" ").join(j.next_sibling.next_sibling.text.split()))
            W = True
        if j.text == "Star:" or j.text == "Stars:":
            data[i].append((" ").join(j.next_sibling.next_sibling.text.split()))
            S = True
        if j.text == "Budget:":
            data[i].append((" ").join(j.next_sibling.split()))
            B = True
        if j.text == "Opening Weekend:":
            data[i].append((" ").join(j.next_sibling.split()))
            O = True
        if j.text == "Gross:":
            data[i].append((" ").join(j.next_sibling.split()))
            G = True
        if j.text == "Release Date:":
            data[i].append((" ").join(j.next_sibling.split()))
            R = True
    if not D:
        data[i].insert(1, np.nan)
    if not W:
        data[i].insert(2, np.nan)
    if not S:
        data[i].insert(3, np.nan)
    if not R:
        data[i].insert(4, np.nan)
    if not B:
        data[i].insert(5, np.nan)
    if not O:
        data[i].insert(6, np.nan)
    if not G:
        data[i].insert(7, np.nan)

        
browser.close()
    


### Uncleaned Dataset

In [305]:
df = pd.DataFrame( data, index = np.arange(n), columns = columns )
df.head()

,Name,Director,Writer,Star,Release.Date,Budget,Opening.Weekend,Gross
0,Drag Me to Hell,Sam Raimi,"Sam Raimi,","Alison Lohman,",29 May 2009 (USA),"$30,000,000","£1,907,731 (UK)","$42,100,625"
1,Ginger Snaps,John Fawcett,"Karen Walton (story),","Emily Perkins,",11 May 2001 (Canada),"$5,000,000","£27,049 (UK)","$2,554"
2,Monsoon Wedding,Mira Nair,Sabrina Dhawan,"Naseeruddin Shah,",26 April 2002 (USA),"INR 7,000,000","£130,466 (UK)","$13,876,974"
3,The Departed,Martin Scorsese,"William Monahan (screenplay),","Leonardo DiCaprio,",6 October 2006 (USA),"$90,000,000","£2,298,313 (UK)","$132,384,315"
4,The Horse Whisperer,Robert Redford,"Nick Evans (novel) (as Nicholas Evans),","Robert Redford,",15 May 1998 (USA),"$60,000,000","$13,685,488 (USA)","$75,383,563"


### Cleaning Data
In the cell below:

- I remove any phrases in parenthesis at the end of a name, so that all people with the same names will have the same spelling. 

- I remove the phrases in parenthesis after the release data, telling where the movie was first released. 

- I remove any commas at the end of names, and also in monetary amount to make it easier to convert them to ints later. 

- For each currency, I remove the symbol for the currency in place it in a new column that corresponds to opening weekend, and budget. 

- Each date is formated uniformly with pandas.to_datetime()

***
Note: Because all gross profits are listed in USD I will not have to worry about that column
***

In [306]:
df1=df
def remove_parenthesis(x):
    x=[str(i).split("(")[0] for i in x]
    return pd.Series(x)
def remove_comma(x):
    l=[]
    for i in x:
        l.append(("").join(str(i).split(",")))
    return pd.Series(l)
df1 = df1.apply(remove_parenthesis, axis = 1)
df1 = df1.apply(remove_comma, axis = 1)
df1[4] = df1[4].apply(lambda x: pd.to_datetime(str(x)))
def _type(x):
    if x == "nan":
        return x
    xs=x.split()
    if len(xs)>1:
        return xs[0]
    else:
        return x[0]
def _int(x):
    if x == "nan":
        return x
    xs=x.split()
    if len(xs)>1:
        return int(xs[1])
    else:
        return int(x[1:])
df1[8] = df1[5].apply(_type)
df1[9] = df1[6].apply(_type)
df1[5] = df1[5].apply(_int)
df1[6] = df1[6].apply(_int)
df1[7] = df1[7].apply(_int)
#column 7 is all in USD
#column 6 units described by column 9
#column 5 units described by column 8

### Finding Data to Convert Currency
In the cell below:
- I read in a dataset that I found that has values for the currency rates from dates from 1971 to 2017, for the exchange rate from USD to 26 different currencies

In [307]:
e_r_columns =["Dates","€","£","BRL","CNY", "DKK", "INR", "JPY", "KRW", "MYR", "MXN", "NOK", "SEK", "ZAR", "SGD", "CHF", "TWD", "THB", "VEB","NBDI","NMC","NOI","AUD", "NZD", "CAD", "HKD", "LKR"]
exchange_rates = pd.read_csv("exchange_rates.csv", skiprows = [0,1,2,4,5,6])#, index_col= e_r_columns)
exchange_rates.columns = e_r_columns
exchange_rates["Dates"] = exchange_rates["Dates"].apply(lambda x: pd.to_datetime(str(x)))
exchange_rates.tail()

,Dates,€,£,BRL,CNY,DKK,INR,JPY,KRW,MYR,...,THB,VEB,NBDI,NMC,NOI,AUD,NZD,CAD,HKD,LKR
12164,2017-08-21,1.1814,1.2903,3.1436,6.6630,6.2959,64.1000,108.8900,1137.1400,4.2855,...,33.2400,9.9750,119.1024,88.0441,152.4720,0.7936,0.7326,1.2587,7.8252,153.1000
12165,2017-08-22,1.1762,1.2834,3.1505,6.6600,6.3233,64.0700,109.4000,1131.7900,4.2790,...,33.2400,9.9750,119.1862,88.2744,152.3615,0.7913,0.7292,1.2559,7.8263,152.9000
12166,2017-08-23,1.1802,1.2787,3.1521,6.6569,6.3019,64.0400,109.1400,1128.5900,4.2780,...,33.3400,9.9750,119.1464,88.1510,152.4319,0.7893,0.7208,1.2558,7.8259,152.9000
12167,2017-08-24,1.1801,1.2802,3.1428,6.6600,6.3018,64.0400,109.3400,1127.8000,4.2780,...,33.3400,9.9750,119.0992,88.0944,152.3997,0.7905,0.7215,1.2528,7.8239,152.9000
12168,2017-08-25,1.1874,1.2873,3.1463,6.6465,6.2655,64.0000,109.2400,1121.7300,4.2715,...,33.2500,9.9750,118.6921,87.7097,151.9869,0.7929,0.7232,1.2494,7.8215,152.9000


### Making Dataset Uniform
In the cell below:
- I use the previous data set to convert each currency that isn't in USD to USD.

In [308]:
def change_to_USD(x):
    
    date = x[4]
    
    if x[8] != "$" and x[8] != "nan":
        try:
            cur_rate = exchange_rates[x[8]][exchange_rates["Dates"] == date].values[0]
            x[5] = int(x[5]) / float(cur_rate)
        except:
            x[5] = "nan"
        
    elif x[9] != "$" and x[9] != "nan":
        try:
            cur_rate = exchange_rates[x[9]][exchange_rates["Dates"] == date].values[0]
            x[6] = int(x[6]) / float(cur_rate)
        except:
            x[5] = "nan"
        
    return x
        
df1 = df1.apply(change_to_USD, axis = 1)

### Data Cleaned and Uniform
In the cell below:
- I set my original dataset to the dataset that I have been manipulating so they are both the same

In [309]:
del df1[8]
del df1[9]
df = df1
df.columns = columns
df.head(20)

,Name,Director,Writer,Star,Release.Date,Budget,Opening.Weekend,Gross
0,Drag Me to Hell,Sam Raimi,Sam Raimi,Alison Lohman,2009-05-29,30000000,1.18053e+06,42100625
1,Ginger Snaps,John Fawcett,Karen Walton,Emily Perkins,2001-05-11,5000000,19079.5,2554
2,Monsoon Wedding,Mira Nair,Sabrina Dhawan,Naseeruddin Shah,2002-04-26,142886,130466,13876974
3,The Departed,Martin Scorsese,William Monahan,Leonardo DiCaprio,2006-10-06,90000000,1.22826e+06,132384315
4,The Horse Whisperer,Robert Redford,Nick Evans,Robert Redford,1998-05-15,60000000,13685488,75383563
5,Your Name.,Leonardo Hwan,Leonardo Hwan,Leonardo Hwan,2017-08-18,nan,nan,nan
6,Snow White and the Seven Dwarfs,William Cottrell,Jacob Grimm,Adriana Caselotti,1938-02-04,2000000,7514749,184925485
7,The Battle of Algiers,Gillo Pontecorvo,Franco Solinas,Brahim Hadjadj,1967-09-20,800000,13536,55908
8,The Punk Singer,Sini Anderson,nan,Kathleen Hanna,2013-03-10,nan,24729,121418
9,Spy Kids,Robert Rodriguez,Robert Rodriguez,Alexa PenaVega,2001-03-30,35000000,799739,112719001


### Units
For the data set above:
- Dates have been uniformly written, using pands.to_datetime
- All currency is in USD

***
Note: It may be misleading that some entries in the opening weekends have a much larger value than the gross profit. This is because the opening weekends column describes all the money spent on that movie during the opening weekend. This is different from the gross, because the gross describes the profit after everyone has been paid.
***

### Data should be scraped and cleaned:
- Different spellings of the same name should be merged
- Variations in the reporting and presentation of the data should be uniformized.
- The units for each measurement should be clearly identified (and appropriately uniformized)
- Data in a given column should all be the same type.
- Missing data is sometimes recorded as 0 instead of NaN.  Fix that.
- Nonnumbers (like zip codes and phone numbers) should not be treated as numbers.
- whitespace and date formats etc should be consistent 


### Identify potential problems with the data:
Evaluate the source
- is it reliable or not? why? 
Evaluate potential biases and other problems with the data.
- What will you do to deal with these problems?
- Are there missing results or other things that look wrong? why? 
- what will you do to deal with these? 
Values that are out of range must be identified and removed or corrected.
Evaluate the suitability of your data for answering the questions in the proposal.  
If it is not suitable, adjust the questions or acquire new data as necessary.
Revise any other aspects of your proposal in light of what you have learned from examining the data.